In [1]:
import numpy as np
import pandas as pd 
import jieba 
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from collections import Counter
from math import isnan
from sklearn.feature_extraction.text import TfidfTransformer
#txt = pd.read_excel('数据比较好的.xlsx')
txt = pd.read_excel('guo(new_3w).xlsx').drop('空白字段',axis = 1)
Y = txt['标签']
'''
jieba.load_userdict(Y)
stopwords = [line.strip() for line in open('../input/stopword.txt', 'r', encoding='utf-8').readlines()] 
list_des = txt['商品描述'].tolist()
X = []
for i in range(len(list_des)):
    des = list_des[i].replace(' ','')
    seg_list = list(jieba.lcut(des))
    word_list = [] 
    for seg in seg_list:
        if seg not in stopwords:
            word_list.append(seg)
    X.append((' ').join(word_list))
txt['商品描述'] = pd.Series(X)  '''

c:\python3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


"\njieba.load_userdict(Y)\nstopwords = [line.strip() for line in open('../input/stopword.txt', 'r', encoding='utf-8').readlines()] \nlist_des = txt['商品描述'].tolist()\nX = []\nfor i in range(len(list_des)):\n    des = list_des[i].replace(' ','')\n    seg_list = list(jieba.lcut(des))\n    word_list = [] \n    for seg in seg_list:\n        if seg not in stopwords:\n            word_list.append(seg)\n    X.append((' ').join(word_list))\ntxt['商品描述'] = pd.Series(X)  "

In [2]:
for i in range(len(txt)):
    length = len(txt.loc[i]['商品描述'].split())
    if length >15:
        txt.drop(index = i,inplace=True)

In [3]:
frame = pd.DataFrame(columns=['商品描述','标签'])
dic = Counter(Y)
for i in dic.keys():
    data = txt.loc[txt['标签'] == i]
    '''if dic[i] > 200:
        frame = pd.concat([frame,data[:200]],axis = 0)
        continue
    elif dic[i]>=10:
        frame = pd.concat([frame,txt.loc[txt['标签'] == i]],axis = 0)'''
    if dic[i]>=10:
        frame = pd.concat([frame,txt.loc[txt['标签'] == i]],axis = 0)
frame = frame.reset_index(drop=True)
c_num = len(Counter(frame['标签']).keys())
print('数据量：%d'%frame.shape[0],'分类数目：%d'%len(Counter(frame['标签']).keys()))

数据量：31674 分类数目：511


In [4]:
word_txt = []
for label in Counter(frame['标签']).keys():
    same = frame.loc[frame['标签'] == label]
    word = []
    for i in same['商品描述']:
        word += i.split()
    word_txt.append(word)
from gensim.models import word2vec
#sentences = word2vec.Text8Corpus('分词拿去做word2vect.txt')
model = word2vec.Word2Vec(word_txt,size = 200,window = 2000,min_count = 10)

c:\python3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
token_dictionary = {}
token_index = 0
word2vec_matrix = []
for line in range(len(frame['商品描述'])):
    tokens = frame['商品描述'][line].split()
    del_list = []
    for token in tokens:
        try:
            temp = model[token]
        except:
            del_list.append(token)
            continue
        else:
            if token not in token_dictionary:
                word2vec_matrix.append(list(temp))
                token_dictionary[token] = token_index
                token_index += 1
    for token in del_list:
        tokens.remove(token)
    frame['商品描述'][line] = ' '.join(tokens)
word2vec_matrix.append([0]*200)
token_dictionary['</s>'] = token_index

c:\python3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [6]:
frame_index = pd.DataFrame(columns=['商品描述','标签'])
padding_index = len(token_dictionary) - 1
index = []
for line in frame['商品描述']:
    tokens = line.split()
    line_index = []
    for token in tokens:
        line_index += [token_dictionary[token]]
    for _ in range(len(line_index),15):
        line_index.append(padding_index)
    index.append(line_index)
    
frame_index['商品描述'] = pd.Series(index)
frame_index['标签'] = frame['标签']

In [7]:
from sklearn import preprocessing 
le = preprocessing.LabelEncoder()
y_label = le.fit_transform(frame_index['标签'])

In [8]:
import tensorflow as tf
from tensorflow.contrib import rnn
import random
input_size = 200
timestep_size = 15
hidden_size = 256
layer_num = 2
class_num = c_num
learning_rate = 0.01
embed_dim = 200
sequence_length = 15
with tf.name_scope('inputs'):
    keep_prob = tf.placeholder(tf.float32)
    batch_size = tf.placeholder(tf.int32, [], name='batch_size_input')
    #_X = tf.placeholder(tf.float32,[None,1500])
    y = tf.placeholder(tf.int64,[None])
    #x = tf.reshape(_X,[-1,timestep_size,input_size])
    input_data = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_data') 
with tf.name_scope("embedding_layer"):
    #embedding = tf.get_variable("embedding",[vocabulary_size,embed_dim],dtype=tf.float32)
    embedding = tf.Variable(word2vec_matrix,dtype = tf.float32)
    inputs=tf.nn.embedding_lookup(embedding,input_data)    
    
#inputs = tf.nn.dropout(inputs, keep_prob)
#input_data (batch_size , sequence_length)
#inputs shape : (batch_size , sequence_length , embed_dim)

inputs = tf.transpose(inputs, [1,0,2])
#inputs shape : (sequence_length, batch_size, embed_dim)

inputs = tf.reshape(inputs, [-1, embed_dim])

# 转换成list,里面的每个元素是(batch_size, embed_dim)
inputs = tf.split(inputs, sequence_length, 0)

with tf.name_scope("fw"):
    stacked_rnn_fw = []
    for _ in range(layer_num):
        #fw_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size, forget_bias=1.0, state_is_tuple=True)
        fw_cell = tf.nn.rnn_cell.GRUCell(hidden_size)
        stacked_rnn_fw.append(fw_cell)
    lstm_fw_cell_m = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.MultiRNNCell(stacked_rnn_fw), output_keep_prob= keep_prob)
    #lstm_fw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_fw, state_is_tuple=True)
with tf.name_scope("bw"):
    stacked_rnn_bw = []
    for _ in range(layer_num):
        #bw_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size, forget_bias=1.0, state_is_tuple=True)
        bw_cell = tf.nn.rnn_cell.GRUCell(hidden_size)
        stacked_rnn_bw.append(bw_cell)
    lstm_bw_cell_m = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.MultiRNNCell(stacked_rnn_bw), output_keep_prob= keep_prob)
    #lstm_bw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_bw, state_is_tuple=True)
with tf.name_scope("output"):
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell_m, lstm_bw_cell_m, inputs, dtype=tf.float32)

In [9]:
attention_size = 256
with tf.name_scope('attention'), tf.variable_scope('attention'):
    attention_w = tf.Variable(tf.truncated_normal([2*hidden_size, attention_size], stddev=0.1), name='attention_w')
    attention_b = tf.Variable(tf.constant(0.1, shape=[attention_size]), name='attention_b')
    u_list = []
    for t in range(15):
        u_t = tf.tanh(tf.matmul(outputs[t], attention_w) + attention_b) 
        u_list.append(u_t)
    u_w = tf.Variable(tf.truncated_normal([attention_size, 1], stddev=0.1), name='attention_uw')
    attn_z = []
    for t in range(15):
        z_t = tf.matmul(u_list[t], u_w)
        attn_z.append(z_t)
    # transform to batch_size * sequence_length
    attn_zconcat = tf.concat(attn_z, axis=1)
    alpha = tf.nn.softmax(attn_zconcat)
    # transform to sequence_length * batch_size * 1 , same rank as outputs
    alpha_trans = tf.reshape(tf.transpose(alpha, [1,0]), [15, -1, 1])
    final_output = tf.reduce_sum(outputs * alpha_trans, 0)

In [10]:
with tf.variable_scope("result",reuse = tf.AUTO_REUSE):
    fc_w = tf.Variable(tf.truncated_normal([2*hidden_size, class_num], stddev=0.1), name='fc_w')
    fc_b = tf.Variable(tf.zeros([class_num]), name='fc_b')
    #self.final_output = outputs[-1]
    # 用于分类任务, outputs取最终一个时刻的输出
    logits = tf.matmul(final_output, fc_w) + fc_b
    #logits = tf.matmul(outputs[-1], fc_w) + fc_b
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits+1e-10,labels = y)
    original_cost_function = tf.reduce_mean(loss)
    
    tv = tf.trainable_variables()
    regularization_cost = 0.001* tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv ])
    cost = original_cost_function + regularization_cost

    train_op = tf.train.AdamOptimizer(learning_rate).minimize(original_cost_function)
with tf.name_scope("accuracy"):
    prediction = tf.argmax(logits,1)
    correct_prediction = tf.equal(prediction,y)
    correct_num=tf.reduce_sum(tf.cast(correct_prediction,tf.float32))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32),name="accuracy")

In [11]:
X_train,X_test,Y_train,Y_test = train_test_split(frame_index['商品描述'].tolist(),y_label,test_size = 0.2)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    listnum = []
    Iter = 0
    for i in range(600):
        _batch_size = 500
        if len(listnum)<_batch_size:
            listnum = list(range(len(X_train)))
            Iter += 1
        temp_x =[]
        temp_y = []
        for _ in range(_batch_size):
            j = random.randint(0,len(listnum)-1) #生成一个包括0，len(listnum)-1之间的随机数
            temp_x.append(X_train[listnum[j]])
            temp_y.append(Y_train[listnum[j]])
            del listnum[j]
        # temp_x 是一个1000*1200维度的向量
        if (i)%20 == 0:
            train_accuracy_test = sess.run(accuracy, feed_dict={input_data:X_test, y: Y_test, keep_prob: 1.0, batch_size:len(X_test)})
            train_accuracy_train = sess.run(accuracy, feed_dict={input_data:temp_x, y: temp_y, keep_prob: 1.0, batch_size:len(X_test)})
            print("Iter %d: step %d, training accuracy %g" % ( Iter,(i+1), train_accuracy_test))
            print('训练集表现：',train_accuracy_train)
        sess.run(train_op,feed_dict={input_data:temp_x,y:temp_y,keep_prob: 0.3, batch_size: _batch_size})

Iter 1: step 1, training accuracy 0.00584057
训练集表现： 0.006
Iter 1: step 21, training accuracy 0.72502
训练集表现： 0.726
Iter 1: step 41, training accuracy 0.835043
训练集表现： 0.808
Iter 2: step 61, training accuracy 0.867088
训练集表现： 0.902
Iter 2: step 81, training accuracy 0.889976
训练集表现： 0.902
Iter 3: step 101, training accuracy 0.902762
训练集表现： 0.944
Iter 3: step 121, training accuracy 0.908445
训练集表现： 0.954
Iter 3: step 141, training accuracy 0.913181
训练集表现： 0.95
Iter 4: step 161, training accuracy 0.915233
训练集表现： 0.972
Iter 4: step 181, training accuracy 0.919968
训练集表现： 0.97
Iter 5: step 201, training accuracy 0.9206
训练集表现： 0.976
Iter 5: step 221, training accuracy 0.921389
训练集表现： 0.976
Iter 5: step 241, training accuracy 0.921231
训练集表现： 0.968
Iter 6: step 261, training accuracy 0.917916
训练集表现： 0.976
Iter 6: step 281, training accuracy 0.927545
训练集表现： 0.99
Iter 7: step 301, training accuracy 0.921863
训练集表现： 0.986
Iter 7: step 321, training accuracy 0.919968
训练集表现： 0.992
Iter 7: step 341, traini

In [12]:
frematrix = CountVectorizer().fit_transform(frame['商品描述'])
tfidf = TfidfVectorizer().fit_transform(frame['商品描述'])
lr = LogisticRegression()
X_train,X_test,Y_train,Y_test = train_test_split(tfidf,frame['标签'])
lr.fit(X_train,Y_train)
pre = lr.predict(X_test)
np.mean(Y_test == pre)

0.9006187649955802